In [4]:
import numpy as np
from qpsolvers import solve_qp
# import pandas to help me output result to csv file
import pandas as pd
# define the RBF kernel function
def rbf_kernel(x1,x2,sigma):
    dx = np.linalg.norm((x1-x2), ord=2)**2
    K = np.exp(-dx/(2*sigma**2))
    return K
#SVM starts
def RBF_SVM(trainx, trainy, testx, pos_label , neg_label, C,sig):
    P = np.zeros((trainx.shape[0], trainx.shape[0]))
    for i in range(trainx.shape[0]):
        for j in range(trainx.shape[0]):
            P[i, j] = trainy[i] * trainy[j] * rbf_kernel(trainx[i], trainx[j], sig)
    H = P
    f = -np.ones(trainy.shape)
    Aeq = trainy
    lb = np.zeros(trainx.shape[0])
    ub = C * (np.ones(trainx.shape[0]))
    beq = np.array([0.0])
    alpha = solve_qp(H, f, None, None, Aeq, beq, lb, ub, solver="clarabel")
#   calculate the optimized bias b*   
    N = 0
    bias = 0
    label = []
    for k in range(alpha.size):
        if  alpha[k] > 0 and alpha[k] < C :
            N += 1
            temp = 0
            for i in range(trainx.shape[0]):
                    temp += alpha[i] * trainy[i] * rbf_kernel(trainx[i], trainx[k], sig)
            bias += (1 / trainy[k]) - temp

    if N > 0:
        bias = (bias / N)
    elif N==0:
        label.append(' ')
    
    for j in range(testx.shape[0]):
        d = 0
        for i in range(trainx.shape[0]):
            if alpha[i] > 0:
                d += alpha[i] * trainy[i] * rbf_kernel(trainx[i], testx[j], sig)
        D = d + bias
        if D >= 0:
            label.append(str(pos_label))
        elif D < 0:
            label.append(str(neg_label))
    label = np.array(label)
    return label


In [20]:
import numpy as np
a0 = 1.05**-100
r = 1.05**5
length = 41 
sigmavalues = a0 * np.power(r, np.arange(length))
Cs = [1, 5, 10, 50, 100, 500, 1000]
irisA = []
irisB = []
irisC = []
trainX12 = []
trainX13 = []
trainX23 = []
with open('iris.txt') as f:
    for line in f:
        line = line.strip().replace('  ', ' ').split(' ')
        iris_type = line[-1]
        iris_data = [float(num) for num in line[:-1]]
        if iris_type == '1':
            irisA.append(iris_data)
        elif iris_type == '2':
            irisB.append(iris_data)
        elif iris_type == '3':
            irisC.append(iris_data)
trainX12 = np.array(irisA[:25] + irisB[:25])
trainX13 = np.array(irisA[:25] + irisC[:25])
trainX23 = np.array(irisB[:25] + irisC[:25])
trainY = np.zeros(trainX12.shape[0])
trainY[:25] = 1
trainY[25:] = -1
testset = np.array(irisA[25:]+irisB[25:]+irisC[25:])
testY = np.zeros(testset.shape[0])
testY[:25] ='1'
testY[25:51] = '2'
testY[50:75] = '3'
results_list = []
#Fold no.1
for C in Cs:
    for sig in sigmavalues:
        SVM12 = RBF_SVM(trainX12, trainY, testset, '1', '2',C, sig)
        SVM13 = RBF_SVM(trainX13, trainY, testset, '1', '3',C, sig)
        SVM23 = RBF_SVM(trainX23, trainY, testset, '2', '3',C, sig)
        result = []
        for i in range(testset.shape[0]):
            result.append(SVM12[i])
            result.append(SVM13[i])
            result.append(SVM23[i])
        result = np.array(result)
        result = result.reshape(-1,3)
        final_result = []
        for i in range(0, len(result)):
            decision = {'1': 0, '2': 0, '3': 0}
            decision[str(result[i][0])] += 1
            decision[str(result[i][1])] += 1
            decision[str(result[i][2])] += 1
            max_vote = max(decision.values())
            winners = [key for key, value in decision.items() if value == max_vote]
            if len(winners) == 1:
                final_result.append(int(winners[0]))
                #print(winners[0])
            else:
                # Handle tie cases here, e.g., consider it as an error
                final_result.append('Tied')
        correct_count = 0
        for i in range(len(testset)) :
            if final_result[i] == testY[i]:
                correct_count+=1
            elif final_result[i] == 'Tied':
                correct_count = 0
                print("Parameter C={:.2f} and sigma= {:.6f} got TIED result".format(C, sig))
                break
        else:
            pass
        accuracy = (correct_count / len(testset)) * 100
        #print("C = {:.1f}, while sigma ={:.4f}".format(C,sig))
        #print("The accuracy is: {:.2f}%".format(accuracy))
        results_list.append({'C': C, 'sigma': sig, 'Accuracy': accuracy})

results_df = pd.DataFrame(results_list)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(results_df)
results_df.to_csv('Output_Fold1.csv', index=False)

D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


Parameter C=5.00 and sigma= 0.009705 got TIED result
Parameter C=50.00 and sigma= 0.007604 got TIED result
Parameter C=50.00 and sigma= 0.009705 got TIED result
Parameter C=100.00 and sigma= 0.007604 got TIED result
Parameter C=500.00 and sigma= 0.007604 got TIED result
Parameter C=500.00 and sigma= 0.009705 got TIED result
Parameter C=1000.00 and sigma= 0.007604 got TIED result
        C       sigma   Accuracy
0       1    0.007604  48.000000
1       1    0.009705  36.000000
2       1    0.012387  62.666667
3       1    0.015809  40.000000
4       1    0.020177  42.666667
5       1    0.025752  66.666667
6       1    0.032866  46.666667
7       1    0.041946  60.000000
8       1    0.053536  60.000000
9       1    0.068326  68.000000
10      1    0.087204  72.000000
11      1    0.111297  80.000000
12      1    0.142046  89.333333
13      1    0.181290  92.000000
14      1    0.231377  94.666667
15      1    0.295303  93.333333
16      1    0.376889  94.666667
17      1    0.481017  9

In [19]:
#Fold no.2 
train2_X12 = []
train2_X13 = []
train2_X23 = []
train2_X12 = np.array(irisA[25:] + irisB[25:])
train2_X13 = np.array(irisA[25:] + irisC[25:])
train2_X23 = np.array(irisB[25:] + irisC[25:])
train2_Y = np.zeros(train2_X12.shape[0])
train2_Y[:25] = 1
train2_Y[25:] = -1
testset2 = np.array(irisA[:25]+irisB[:25]+irisC[:25])
test2_Y = np.zeros(testset2.shape[0])
test2_Y[:25] ='1'
test2_Y[25:50] = '2'
test2_Y[50:75] = '3'
results_list2 = []


for C in Cs:
    for sig in sigmavalues:
        SVM2_12 = RBF_SVM(train2_X12, train2_Y, testset2, '1', '2',C, sig)
        SVM2_13 = RBF_SVM(train2_X13, train2_Y, testset2, '1', '3',C, sig)
        SVM2_23 = RBF_SVM(train2_X23, train2_Y, testset2, '2', '3',C, sig)
        result2 = []
        for i in range(testset2.shape[0]):
            result2.append(SVM2_12[i])
            result2.append(SVM2_13[i])
            result2.append(SVM2_23[i])
        result2 = np.array(result2)
        result2 = result2.reshape(-1,3)
        final_result2 = []
        for i in range(0, len(result2)):
            decision = {'1': 0, '2': 0, '3': 0}
            decision[str(result2[i][0])] += 1
            decision[str(result2[i][1])] += 1
            decision[str(result2[i][2])] += 1
            max_vote = max(decision.values())
            winners = [key for key, value in decision.items() if value == max_vote]
            if len(winners) == 1:
                final_result2.append(int(winners[0]))
            else:
                # Handle tie cases here, e.g., consider it as an error
                final_result2.append('Tied')
                
        correct_count2 = 0
        for i in range(len(testset)) :
            if final_result2[i] == testY[i]:
                correct_count2+=1
            elif final_result2[i] == 'Tied':
                correct_count2 = 0
                print("Parameter C={:.2f} and sigma= {:.6f} got TIED result".format(C, sig))
                break
        else:
            pass
        accuracy2 = (correct_count2 / len(testset2)) * 100
        print("C = {:.1f}, while sigma ={:.4f}".format(C,sig))
        print("The accuracy is: {:.2f}%".format(accuracy2))
        results_list2.append({'C': C, 'sigma': sig, 'Accuracy': accuracy2})

results2_df = pd.DataFrame(results_list2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
results2_df.to_csv('Output_Fold2.csv', index=False)

D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


C = 1.0, while sigma =0.0076
The accuracy is: 34.67%
C = 1.0, while sigma =0.0097
The accuracy is: 36.00%
C = 1.0, while sigma =0.0124
The accuracy is: 36.00%
C = 1.0, while sigma =0.0158
The accuracy is: 36.00%
C = 1.0, while sigma =0.0202
The accuracy is: 36.00%
C = 1.0, while sigma =0.0258
The accuracy is: 36.00%
C = 1.0, while sigma =0.0329
The accuracy is: 36.00%
C = 1.0, while sigma =0.0419
The accuracy is: 40.00%
C = 1.0, while sigma =0.0535
The accuracy is: 52.00%
C = 1.0, while sigma =0.0683
The accuracy is: 58.67%
C = 1.0, while sigma =0.0872
The accuracy is: 60.00%
C = 1.0, while sigma =0.1113
The accuracy is: 69.33%
C = 1.0, while sigma =0.1420
The accuracy is: 81.33%
C = 1.0, while sigma =0.1813
The accuracy is: 89.33%
C = 1.0, while sigma =0.2314
The accuracy is: 93.33%
C = 1.0, while sigma =0.2953
The accuracy is: 97.33%
C = 1.0, while sigma =0.3769
The accuracy is: 96.00%
C = 1.0, while sigma =0.4810
The accuracy is: 96.00%
C = 1.0, while sigma =0.6139
The accuracy is: 

C = 50.0, while sigma =11.4674
The accuracy is: 96.00%
C = 50.0, while sigma =14.6356
The accuracy is: 97.33%
C = 50.0, while sigma =18.6792
The accuracy is: 94.67%
C = 50.0, while sigma =23.8399
The accuracy is: 96.00%
C = 50.0, while sigma =30.4264
The accuracy is: 94.67%
C = 50.0, while sigma =38.8327
The accuracy is: 93.33%
C = 50.0, while sigma =49.5614
The accuracy is: 93.33%
C = 50.0, while sigma =63.2544
The accuracy is: 92.00%
C = 50.0, while sigma =80.7304
The accuracy is: 92.00%
C = 50.0, while sigma =103.0347
The accuracy is: 92.00%
C = 50.0, while sigma =131.5013
The accuracy is: 92.00%
C = 100.0, while sigma =0.0076
The accuracy is: 34.67%
C = 100.0, while sigma =0.0097
The accuracy is: 34.67%
C = 100.0, while sigma =0.0124
The accuracy is: 36.00%
C = 100.0, while sigma =0.0158
The accuracy is: 36.00%
C = 100.0, while sigma =0.0202
The accuracy is: 36.00%
C = 100.0, while sigma =0.0258
The accuracy is: 36.00%
C = 100.0, while sigma =0.0329
The accuracy is: 36.00%
C = 100.

D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
D:\Python\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
